<img src="https://www.unir.net/wp-content/uploads/2019/11/Unir_2021_logo.svg" width="240" height="240" align="right"/>

<center><h1>TRABAJO FIN DE MÁSTER</header1></center>

## Importación de librerias necesarias

In [114]:
from pandas import read_csv
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd

In [115]:
#carga del fichero

df = pd.read_csv("df_modelo.csv",sep=",")

# Ver las primeras filas para comprobar
print(df.head())

   FECHA  COD_MUNICIPIO                      MUNICIPIO  POBLACION_TOTAL  \
0   2005          42001                   42001 ABEJAR              392   
1   2005          42003                  42003 ADRADAS               82   
2   2005          42004                   42004 AGREDA             3216   
3   2005          42006                 42006 ALCONABA              171   
4   2005          42007  42007 ALCUBILLA DE AVELLANEDA              183   

   PROP_ENVEJECIMIENTO  TASA_DEMANDA  TASA_PARO  RATIO_DEMANDA_PARO  \
0             0.306122      4.184100   2.928870            1.428571   
1             0.439024      7.142857   7.142857            1.000000   
2             0.266169      5.144033   4.372428            1.176471   
3             0.257310      6.194690   3.539823            1.750000   
4             0.519126      6.024096   6.024096            1.000000   

   SERVICIOS_TURISTICOS  SERVICIOS_SANIDAD  CONSULTORIOS  FARMACIAS  \
0                  16.0                NaN         

In [116]:
print(df.describe())

             FECHA  COD_MUNICIPIO  POBLACION_TOTAL  PROP_ENVEJECIMIENTO  \
count  3660.000000    3660.000000      3660.000000          3654.000000   
mean   2014.500000   42107.759563       500.807650             0.410332   
std       5.767069      64.711584      2975.557187             0.122252   
min    2005.000000   42001.000000         6.000000             0.047619   
25%    2009.750000   42050.000000        36.000000             0.326316   
50%    2014.500000   42106.000000        75.000000             0.406509   
75%    2019.250000   42165.000000       179.000000             0.490566   
max    2024.000000   42219.000000     40750.000000             0.800000   

       TASA_DEMANDA    TASA_PARO  RATIO_DEMANDA_PARO  SERVICIOS_TURISTICOS  \
count   3654.000000  3654.000000         2409.000000           3660.000000   
mean       5.850215     4.258882            1.382300              4.184153   
std        5.883087     4.470152            0.509981             10.437281   
min        0

Conclusiones del EDA: 

Lo primero que observamos es que hay bastante varianza en la mayoría de variables, estando muy lejos el valor minimo del maximo. Esto se produce porque hay un municipio que destaca muy por encima de los demás, que es la capital. Esto se ve muy bien por ejemplo en la variable alumnos, con un minimo de 0 y un maximo de 11800. Por tanto, para el análisis vamos a excluir el municipio de Soria.

Volvemos a recalcular lo mismo: 


In [118]:
#df["COD_MUNICIPIO"].unique

df["COD_MUNICIPIO"] = (
    df["COD_MUNICIPIO"]
    .astype(str)
    .str.strip()
)

df = df[df["COD_MUNICIPIO"] != "42173"]

print(df.describe())

             FECHA  POBLACION_TOTAL  PROP_ENVEJECIMIENTO  TASA_DEMANDA  \
count  3640.000000      3640.000000          3634.000000   3634.000000   
mean   2014.500000       287.406044             0.411495      5.812793   
std       5.767074       750.335778             0.121574      5.872613   
min    2005.000000         6.000000             0.047619      0.000000   
25%    2009.750000        36.000000             0.328063      0.000000   
50%    2014.500000        75.000000             0.407576      4.916644   
75%    2019.250000       177.000000             0.491228      9.090909   
max    2024.000000      6006.000000             0.800000    112.195122   

         TASA_PARO  RATIO_DEMANDA_PARO  SERVICIOS_TURISTICOS  \
count  3634.000000         2389.000000           3640.000000   
mean      4.236620            1.380783              3.585165   
std       4.468038            0.511473              6.523123   
min       0.000000            1.000000              0.000000   
25%       0.0

In [119]:
from ydata_profiling import ProfileReport

# Generar el reporte
profile = ProfileReport(df, title="Reporte EDA", explorative=True)

# Guardar en HTML
profile.to_file("eda_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 18/18 [00:00<00:00, 45.17it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Se identifican los valores faltantes en cada variable explicativa con el objetivo de evaluar la necesidad de aplicar técnicas de imputación o eliminación de observaciones.

Vamos a inspeccionar cuales son las variables con mas numero de datos faltantes:

In [126]:
df.isnull().sum().sort_values(ascending=False)

SERVICIOS_SANIDAD            1456
FARMACIAS                    1456
EMPRESAS                     1274
RATIO_DEMANDA_PARO           1251
CONSULTORIOS                  546
ESCUELAS                       20
PROP_POBLACION_EXTRANJERA       6
PROP_ENVEJECIMIENTO             6
TASA_DEMANDA                    6
TASA_PARO                       6
HOSPITALES                      0
ALUMNOS                         0
FECHA                           0
COD_MUNICIPIO                   0
SERVICIOS_TURISTICOS            0
POBLACION_TOTAL                 0
MUNICIPIO                       0
DISTANCIA_CAPITAL               0
dtype: int64

In [128]:
variables_na = [
    "SERVICIOS_SANIDAD",
    "EMPRESAS",
    "RATIO_DEMANDA_PARO"
]

na_por_anio = (
    df.groupby("FECHA")[variables_na]
      .apply(lambda x: x.isnull().sum())
      .reset_index()
)

na_por_anio


,FECHA,SERVICIOS_SANIDAD,EMPRESAS,RATIO_DEMANDA_PARO
0,2005,182,182,75
1,2006,182,182,76
2,2007,182,182,74
3,2008,182,182,71
4,2009,182,182,63
5,2010,182,182,52
6,2011,0,182,54
7,2012,0,0,47
8,2013,0,0,52
9,2014,182,0,55


Vemos que los datos faltantes están el periodo anterior a 2012. por lo tanto, vamos a restringuir el modelo todavia más, de 2012 a 2024. 

In [130]:
df = df[df["FECHA"].between(2012,2024)]

In [131]:
df.isnull().sum().sort_values(ascending=False)

RATIO_DEMANDA_PARO           786
SERVICIOS_SANIDAD            364
FARMACIAS                    364
ESCUELAS                      13
PROP_POBLACION_EXTRANJERA      6
PROP_ENVEJECIMIENTO            6
TASA_DEMANDA                   6
TASA_PARO                      6
EMPRESAS                       0
ALUMNOS                        0
HOSPITALES                     0
FECHA                          0
CONSULTORIOS                   0
COD_MUNICIPIO                  0
SERVICIOS_TURISTICOS           0
POBLACION_TOTAL                0
MUNICIPIO                      0
DISTANCIA_CAPITAL              0
dtype: int64

Para escuelas, prop_envejecimiento, tasa_demanda, tasa_paro, prop_poblacion_extranjera, sustituimos los NA con ceros.

Para servicios_sanidad, el NA viene porque uno de los 3 indicadores que se suma es NA (consultorios, farmacias y hospitales). Para corregir esto, vamos a sustituir los NA por ceros en estas tres variables y despues volvemos a calcular la suma 

In [148]:
cols = [
    "CONSULTORIOS",
    "FARMACIAS",
    "HOSPITALES",
    "ESCUELAS",
    "PROP_ENVEJECIMIENTO",
    "TASA_DEMANDA",
    "TASA_PARO",
    "PROP_POBLACION_EXTRANJERA"
]

df = df.copy()

df[cols] = df[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

cols_sanidad = [
    "CONSULTORIOS",
    "FARMACIAS",
    "HOSPITALES"
]

df.loc[:, "SERVICIOS_SANIDAD"] = df.loc[:, cols_sanidad].sum(axis=1)

Volvemos a comprobar valores faltantes

In [151]:
df.isnull().sum().sort_values(ascending=False)

RATIO_DEMANDA_PARO           786
FECHA                          0
CONSULTORIOS                   0
PROP_POBLACION_EXTRANJERA      0
EMPRESAS                       0
ALUMNOS                        0
ESCUELAS                       0
HOSPITALES                     0
FARMACIAS                      0
SERVICIOS_SANIDAD              0
COD_MUNICIPIO                  0
SERVICIOS_TURISTICOS           0
TASA_PARO                      0
TASA_DEMANDA                   0
PROP_ENVEJECIMIENTO            0
POBLACION_TOTAL                0
MUNICIPIO                      0
DISTANCIA_CAPITAL              0
dtype: int64

la columna ratio_demanda_paro la eliminamos del modelo, y eliminamos consultorios, farmacias y hospitales, ya que ya estan en la variable servicios_sanidad. 

Escuelas y alumnos son dos variables muy relacionadas entre ellas, como se puede ver en la matriz de correlacion. Tiene mas sentido que lo que influya en la poblacion sea el numero de escuelas, independientemente de los alumnos que haya en ella, por lo tanto, eliminamos alumnos y nos quedamos con escuelas. 

In [179]:
df_modelo_POST_EDA = df.drop(columns=[
    "CONSULTORIOS",
    "FARMACIAS",
    "HOSPITALES",
    "RATIO_DEMANDA_PARO",
    "ALUMNOS"
])

df_modelo_POST_EDA.isnull().sum().sort_values(ascending=False)

FECHA                        0
COD_MUNICIPIO                0
MUNICIPIO                    0
POBLACION_TOTAL              0
PROP_ENVEJECIMIENTO          0
TASA_DEMANDA                 0
TASA_PARO                    0
SERVICIOS_TURISTICOS         0
SERVICIOS_SANIDAD            0
ESCUELAS                     0
EMPRESAS                     0
PROP_POBLACION_EXTRANJERA    0
DISTANCIA_CAPITAL            0
dtype: int64

In [180]:
df_modelo_POST_EDA.to_csv(
    "df_modelo_POST_EDA.csv",
    index=False,
    encoding="utf-8"
)